In [60]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import transforms

In [125]:
transform = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

In [126]:
training_data = datasets.Flowers102(root = 'data', split = 'train', download = True, transform = transform)
testing_data = datasets.Flowers102(root = 'data', split = 'test', download = True, transform = transform)

In [127]:
trainingLoader = DataLoader(training_data, batch_size=16)
testingLoader = DataLoader(testing_data, batch_size=16)

In [191]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(3,3)
        self.conv2 = nn.Conv2d(16,32,5)
        self.conv3 = nn.Conv2d(32,128,5)
        self.fc1 = nn.Linear(512,256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 102)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [192]:
network = NeuralNetwork()

In [193]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr = 0.01)

In [196]:
epoch = 5

for i in range(epoch):
    running_loss = 0
    for j, data in enumerate(trainingLoader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = network(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        print(f'[{i + 1}, {j + 1:5d}] loss: {running_loss}')
        running_loss = 0

print('Finished Training')

[1,     1] loss: 4.60264253616333
[1,     2] loss: 4.638364315032959
[1,     3] loss: 4.592617511749268
[1,     4] loss: 4.644452095031738
[1,     5] loss: 4.660061359405518
[1,     6] loss: 4.631757736206055
[1,     7] loss: 4.563920021057129
[1,     8] loss: 4.625548839569092
[1,     9] loss: 4.598842620849609
[1,    10] loss: 4.6224799156188965
[1,    11] loss: 4.568737030029297
[1,    12] loss: 4.60933780670166
[1,    13] loss: 4.616479396820068
[1,    14] loss: 4.600872039794922
[1,    15] loss: 4.683802127838135
[1,    16] loss: 4.674005031585693
[1,    17] loss: 4.6639790534973145
[1,    18] loss: 4.651022911071777
[1,    19] loss: 4.555922031402588
[1,    20] loss: 4.663208961486816
[1,    21] loss: 4.632880210876465
[1,    22] loss: 4.5813889503479
[1,    23] loss: 4.638350963592529
[1,    24] loss: 4.665174961090088
[1,    25] loss: 4.603099822998047
[1,    26] loss: 4.637110710144043


KeyboardInterrupt: 

In [172]:
correct = 0
total = 0

for data in testingLoader:
    images, labels = data
    output = network(images)
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print(f"Network accuracy = {100 * correct / total}%")

Network accuracy = 0.601723857537811%
